In [31]:
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets

In [8]:
class Generator(torch.nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        self.encoder0 = nn.Sequential(
            nn.Conv2d(13, 64, (4,4), stride=(2,2), padding=0),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(negative_slope=0.2)
        )
        
        self.encoder1 = nn.Sequential(
            nn.Conv2d(64, 128, (4,4), stride=(2,2), padding=0),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(negative_slope=0.2)
        )
        
        self.hidden0 = nn.Sequential(
            nn.Conv2d(128, 512, (4,4), stride=(2,2), padding=(0)),
            nn.ReLU()
        )
        
        self.decoder0 = nn.Sequential(
            nn.ConvTranspose2d(512, 128, (4,4), stride=(2,2)),
            nn.BatchNorm2d(128),
            nn.Dropout()
            # CONCATENATE OUTPUT WITH OUTPUT OF ENC1
        )
        
        self.decoder1 = nn.Sequential(
            nn.ConvTranspose2d(128, 64, (4,4), stride=(2,2)),
            nn.BatchNorm2d(64),
            nn.Dropout()
            # CONCATENATE OUTPUT WITH OUTPUT OF ENC0
        )
        
        self.hidden1 = nn.Sequential(
            nn.ConvTranspose2d(64, 13, (4,4), stride=(2,2)),
            nn.Softmax()
        )
        
    def forward(self, x):
        x0 = x.copy()
        x1 = self.encoder0(x)
        x2 = self.encoder1(x1)
        x = self.hidden0(x2)
        x = self.decoder0(x)
        # concatenate output of decoder0 with output of encoder1
        x = torch.cat((x,x2))
        x = self.decoder1(x)
        # concatenate output of decoder1 with output of encoder0
        x = torch.cat((x,x1))
        x = self.hidden1(x)
        out_tensor = torch.cat((x0,x))
        return out_tensor

In [9]:
class Discriminator(torch.nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.hidden0 = nn.Sequential( 
            nn.Conv2d(13, 64, (4,4), stride=(2,2), padding=0),
            nn.LeakyReLU(negative_slope=0.2)
        )
        self.hidden1 = nn.Sequential(
            nn.Conv2d(64, 128, (4,4), stride=(2,2), padding=0),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(negative_slope=0.2)
        )
        self.hidden2 = nn.Sequential(
            nn.Conv2d(128, 256, (4,4), stride=(2,2), padding=0),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(negative_slope=0.2)
        )
        self.hidden3 = nn.Sequential(
            nn.Conv2d(256, 512, (4,4), stride=(2,2), padding=0),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(negative_slope=0.2)
        )
        self.hidden4 = nn.Sequential(
            nn.Conv2d(512, 512, (4,4), padding=0),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(negative_slope=0.2)
        )
        self.out = nn.Sequential(
            # Output is a single variable representing probability between 0 and 1 that the input is real
            nn.Conv2d(512, 1, (4,4), padding=0),
            nn.Sigmoid()
        )

    def forward(self, x):
        """Note: assuming x is already concatenated in form [initial board, next board]."""
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.hidden3(x)
        x = self.hidden4(x)
        x = self.out(x)
        return x

In [10]:
discriminator = Discriminator()
generator = Generator()

# Optimizers
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002)

# Loss function
loss = nn.BCELoss()

# Number of steps to apply to the discriminator
d_steps = 1  # In Goodfellow et. al 2014 this variable is assigned to 1
# Number of epochs
num_epochs = 200

In [11]:
# changed torch.ones(size,1) to torch.ones(1,1) since output of model is a single number
def real_data_target():
    '''
    Tensor containing ones, with shape = size
    '''
    data = Variable(torch.ones(1, 1))
    if torch.cuda.is_available(): return data.cuda()
    return data

def fake_data_target():
    '''
    Tensor containing zeros, with shape = size
    '''
    data = Variable(torch.zeros(1, 1))
    if torch.cuda.is_available(): return data.cuda()
    return data

In [12]:
def train_discriminator(optimizer, real_data, fake_data):
    # Reset gradients
    optimizer.zero_grad()
    
    # 1.1 Train on Real Data
    prediction_real = discriminator(real_data)
    # Calculate error and backpropagate
    error_real = loss(prediction_real, real_data_target())
    error_real.backward()

    # 1.2 Train on Fake Data
    prediction_fake = discriminator(fake_data)
    # Calculate error and backpropagate
    error_fake = loss(prediction_fake, fake_data_target())
    error_fake.backward()
    
    # 1.3 Update weights with gradients
    optimizer.step()
    
    # Return error
    return error_real + error_fake, prediction_real, prediction_fake

def train_generator(optimizer, fake_data):
    # 2. Train Generator
    # Reset gradients
    optimizer.zero_grad()
    # Sample noise and generate fake data
    prediction = discriminator(fake_data)
    # Calculate error and backpropagate
    error = loss(prediction, real_data_target())
    error.backward()
    # Update weights with gradients
    optimizer.step()
    # Return error
    return error

In [21]:
def gen_real_samples(X,y):
    """Returns a list of tensors in the form [initial board, next board]"""
    data = []
    for i in range(X.shape[0]):
        x_elt = torch.Tensor(X[i])
        y_elt = torch.Tensor(y[i])
        elt = torch.cat((x_elt, y_elt))
        data.append(elt)
    return data

# Load data

In [27]:
import os
import numpy as np
from utils import xy_split, generate_random_boards

#all_data = generate_random_boards()

with open('random_data.npy', 'rb') as f:
    all_data = np.load(f)

X,y = xy_split(all_data, True)

data = gen_real_samples(X,y)

# Create loader with data, so that we can iterate over it
data_loader = torch.utils.data.DataLoader(data, batch_size=100, shuffle=True)
# Num batches
num_batches = len(data_loader)

183 183


In [34]:
def mnist_data():
    compose = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize([0.5], [0.5])
        ])
    out_dir = '{}/dataset'.format("trying_something")
    return datasets.MNIST(root=out_dir, train=True, transform=compose, download=True)

In [35]:
data2 = mnist_data()

0.1%

100.0%


Extracting trying_something/dataset/MNIST/raw/train-images-idx3-ubyte.gz to trying_something/dataset/MNIST/raw



102.8%


Extracting trying_something/dataset/MNIST/raw/train-labels-idx1-ubyte.gz to trying_something/dataset/MNIST/raw



...

Extracting trying_something/dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to trying_something/dataset/MNIST/raw



112.7%


Extracting trying_something/dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to trying_something/dataset/MNIST/raw

Processing...


/mnt/c/Users/Josh Silverberg/Desktop/MDST/mdchesst/team1/env/lib/python3.8/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


AttributeError: 'MNIST' object has no attribute 'shape'

In [37]:

logger = Logger(model_name='VGAN', data_name='MNIST')

for epoch in range(num_epochs):
    for n_batch, (real_batch,_) in enumerate(data_loader):
        print(_)
        '''
        # 1. Train Discriminator
        real_data = Variable(images_to_vectors(real_batch))
        if torch.cuda.is_available(): real_data = real_data.cuda()
        # Generate fake data - form [initial board, next board]
        fake_data = generator(noise(real_data.size(0))).detach()
        # Train D
        d_error, d_pred_real, d_pred_fake = train_discriminator(d_optimizer,
                                                                real_data, fake_data)

        # 2. Train Generator
        # Generate fake data
        fake_data = generator(noise(real_batch.size(0)))
        # Preprocess fake data (merge with previous board)
        # Train G
        g_error = train_generator(g_optimizer, fake_data)
        # Log error
        logger.log(d_error, g_error, epoch, n_batch, num_batches)
        '''
        
        # Display Progress
        if (n_batch) % 100 == 0:
            display.clear_output(True)
            # Display Images
            test_images = vectors_to_images(generator(test_noise)).data.cpu()
            logger.log_images(test_images, num_test_samples, epoch, n_batch, num_batches);
            # Display status Logs
            logger.display_status(
                epoch, num_epochs, n_batch, num_batches,
                d_error, g_error, d_pred_real, d_pred_fake
            )
        # Model Checkpoints
        logger.save_models(generator, discriminator, epoch)

NameError: name 'Logger' is not defined